In [6]:
from pathlib import Path

import polars as pl

from stocksense.database import DatabaseHandler

DATE = "2023-12-01"
REPORT_DIR = Path("../reports/scores")

In [7]:
stock_info = DatabaseHandler().fetch_stock()
df = pl.read_csv(REPORT_DIR / f"scores_{DATE}.csv")
df = df.join(stock_info, on="tic", how="left")
df

2024-12-14 16:38:06.574 | SUCCESS  | stocksense.database.schema:create_tables:121 - Tables created successfully


tic,adj_close,pe,saleq_yoy,fwd_return,excess_return,risk_return,pred_risk_return_3Q,rank_risk_return_3Q,pred_risk_return_4Q,rank_risk_return_4Q,avg_score,name,sector,date_added,date_removed
str,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64,str,str,date,date
"""SEE""",33.487507,15.771561,-1.328192,6.042746,-13.100373,2.886573,5.6585712,9,9.603384,8,8.5,"""Sealed Air""","""Materials""",null,2023-12-18
"""WBA""",19.217991,-5.8298,9.16207,-38.549164,-57.692283,-14.640517,5.6297507,11,9.533597,18,14.5,"""Walgreens Boots Alliance""","""Consumer Staples""",1979-12-31,null
"""FMC""",53.697098,14.045465,-28.703166,8.275038,-10.868082,2.731069,5.662194,8,9.511567,23,15.5,"""FMC Corporation""","""Materials""",2009-08-19,null
"""PSA""",255.414917,23.092956,5.117858,16.228237,-2.914882,10.844826,5.594103,25,9.599665,9,17.0,"""Public Storage""","""Real Estate""",2005-08-19,null
"""LUV""",26.062613,31.94777,4.903537,3.950928,-15.192191,1.546787,5.621783,15,9.521734,21,18.0,"""Southwest Airlines""","""Industrials""",1994-07-01,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""NOW""",690.789978,89.312698,24.959039,11.744603,-7.398516,5.550115,4.9748783,432,8.223453,452,442.0,"""ServiceNow""","""Information Technology""",2019-11-21,null
"""NVDA""",46.750851,61.055955,200.0,158.87167,100.0,52.031133,4.9722304,433,8.100908,456,444.5,"""Nvidia""","""Information Technology""",2001-11-30,null
"""ANET""",216.639999,35.442916,28.267736,55.722429,36.57931,22.902473,4.84744,448,8.336229,443,445.5,"""Arista Networks""","""Information Technology""",2018-08-28,null


In [10]:
n = 40
top = df.head(n)
bottom = df.tail(n)

top_freturn = top.select(pl.col("risk_return")).mean().item()
bottom_freturn = bottom.select(pl.col("risk_return")).mean().item()

# Calculate hit rates (% of stocks with positive returns)
top_hits = top.select(pl.col("risk_return") > 0).sum().item()
bottom_hits = bottom.select(pl.col("risk_return") > 0).sum().item()

top_hitrate = (top_hits / n) * 100
bottom_hitrate = (bottom_hits / n) * 100


print(f"\nDATE {DATE}")
print(f"Average top return: {top_freturn:.2f}% ({top_hitrate:.1f})")
print(f"Average bottom return: {bottom_freturn:.2f}% ({bottom_hitrate:.1f})")


DATE 2023-12-01
Average top return: 4.36% (70.0)
Average bottom return: 10.92% (82.5)


In [4]:
round(0.34539, 3)

0.345